In [1]:
!pip show certifi

Name: certifi
Version: 2022.9.24
Summary: Python package for providing Mozilla's CA Bundle.
Home-page: https://github.com/certifi/python-certifi
Author: Kenneth Reitz
Author-email: me@kennethreitz.com
License: MPL-2.0
Location: c:\programdata\miniconda3\envs\pythonproject\lib\site-packages
Requires: 
Required-by: requests


In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options


In [3]:
# view는 스크롤을 계속 내리는 코드를 작성해줘야 됨
# 스크롤을 끝까지 내린 후 링크를 모두 수집
# 수집한 링크를 하나씩 들어가서 데이터 수집
# 월별로 최대 1000개 크롤링 가능


In [6]:
# 크롤링 시작과 끝 날짜(달 별로 크롤링)
first_days = pd.date_range('2021/01/01','2021/01/31', freq='MS')   # 각 달의 첫 날
last_days = pd.date_range('2021/01/01','2021/01/31', freq='M')  # 각 달의 마지막 날

keyword_list = ['술집','인스타','이색','감성'] # 검색키워드 추가

In [7]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드

chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치


In [10]:
# 블로그 주소 수집
blog_url_list = []

# 네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)

    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)

    driver.get(url)
    time.sleep(3)

    url_want = 990  # 월별 1000개까지만 가능하기 때문에 개수 설정

    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭

    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()

            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            scroll = (url_want/30)-1    # 한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0.document.body.scrollHeight)')  # 0부터 끝까지 스크롤 내림
                time.sleep(random.uniform(1,1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                if(new_height == last_height):
                    break

                last_height = new_height

            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_ = 'api_txt_lines total_tit')   # 첫번째 view 제목 class

            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) # 주소 수집
            
            print(f'{keyword} 키워드 {first_days.year[k]}년 {first_days.month[k]}월', len(blog_url), '개 블로그 url 주소 수집') 

            driver.execute_script('window.scrollTo(0,0)')   # 스크롤 밑으로 내려가지 않게 해줌

        except:
            pass




C:\Users\user\AppData\Local\Temp\ipykernel_12256\1421537061.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


In [9]:
print(blog_url_list)

[]
